In [1]:
# Algoritmo base
# https://www.cs.utexas.edu/~flame/Notes/NotesOnCholReal.pdf

# Alterações feitas:

# A matriz L = zero(A) está sendo criada fora da função
# As alterações estão sendo feitas na própria matriz A
# Utilizar .= e .-=, operações in-place, otimizaram a alocação
# import Pkg
# Pkg.add("BenchmarkTools")

In [2]:
using(LinearAlgebra)
using BenchmarkTools

In [86]:
##### Nível 2

# Algoritmo do livro (applied numerical linear algebra)
function  Cholesky_factorize(A)
    
    # Número de linhas (1 alloc)
    m = size(A, 1)
    
    L = zeros(Float64, m, m)
    
    # Loop principal
    for j in 1:m
        
        # Somatório de Ljk ao quadrado
        sumLjk::Float64 = 0.0
        
        for k in 1:j-1
            sumLjk += L[j, k]*L[j, k]
        end

        # Alterações na diagonal de L
         L[j, j] = sqrt(A[j, j] - sumLjk)

        # Segundo loop principal
        for i in j+1:m
            
            # Somatório do produto Lik e Ljk
            sumLikLjk::Float64 = 0.0
             for k in 1:j-1
                sumLikLjk += L[i, k]*L[j, k]
            end

            # Alterações
             L[i, j] = (A[i, j] - sumLikLjk)/L[j, j]
        end
    end

    return L
end

Cholesky_factorize (generic function with 1 method)

In [90]:
# TESTANDO NÍVEL 2 - TEMPO E ALOCAÇÃO

n = 10
X = randn(n, n)
A = X*X'
B = copy(A)

@time Cholesky_factorize(A)
@time cholesky(A)

print("------------------------------------------------")

  0.000010 seconds (1 allocation: 896 bytes)
  0.000021 seconds (5 allocations: 1008 bytes)
------------------------------------------------

In [130]:
### Nível 3 ######

function Cholesky_by_blocks(A, block_size)
    # Número de linhas
    n = size(A)[1]
    for j in 1:block_size:n
        b = min(n - j + 1, block_size)
        # Realizando as alterações na própria matriz A, poupa alocações
        A[j:j+b-1, j:j+b-1] .= cholesky(Symmetric(A[j:j+b-1, j:j+b-1])).L
        A[j+b:n, j:j+b-1] /= A[j:j+b-1, j:j+b-1]'
        A[j+b:n, j+b:n] .-= A[j+b:n, j:j+b-1]*A[j+b:n, j:j+b-1]'

    end
    return tril(A)
end

Cholesky_by_blocks (generic function with 2 methods)

In [140]:
# TESTANDO NÍVEL 3 - ALOCAÇÃO E TEMPO - TESTE 1
n = 800
X = randn(n, n)
size_block = 512
 
A = X*X'

@time Cholesky_factorize(A)
@time cholesky(A)
@time Cholesky_by_blocks(A, size_block) # Essa deve ficar por último pois altera A
print("----------------------------------------------------------------------------------")

  0.127589 seconds (2 allocations: 4.883 MiB, 2.35% gc time)
  0.006350 seconds (6 allocations: 4.883 MiB)
  0.011612 seconds (53 allocations: 22.307 MiB)
----------------------------------------------------------------------------------

In [695]:
# TESTANDO NÍVEL 3 - ALOCAÇÃO E TEMPO - TESTE 2
n = 3000
X = randn(n, n)
size_block = 2048
A = X*X'

@time Cholesky_factorize(A)
@time cholesky(A)
@time Cholesky_by_blocks(A, size_block) # Essa deve ficar por último pois altera A
print("----------------------------------------------------------------------------------")

 78.980518 seconds (3 allocations: 68.665 MiB)
  2.625674 seconds (4 allocations: 68.665 MiB)
 38.679976 seconds (49 allocations: 273.614 MiB, 8.69% gc time)
----------------------------------------------------------------------------------

In [113]:
#### COMPARAÇÃO DOS RESULTADOS
n = 5
X = randn(5, 5)
A =X*X'

display(Cholesky_factorize(A))
display(cholesky(A).L)
display(Cholesky_by_blocks(A, 2))

5×5 Array{Float64,2}:
 2.47912    0.0         0.0       0.0      0.0
 0.188601   2.81673     0.0       0.0      0.0
 0.0056648  1.85632     1.11233   0.0      0.0
 1.83752    0.0364813  -0.106363  1.02072  0.0
 1.46324    0.678428    2.84908   0.60267  0.0500096

5×5 LowerTriangular{Float64,Array{Float64,2}}:
 2.47912     ⋅           ⋅         ⋅        ⋅ 
 0.188601   2.81673      ⋅         ⋅        ⋅ 
 0.0056648  1.85632     1.11233    ⋅        ⋅ 
 1.83752    0.0364813  -0.106363  1.02072   ⋅ 
 1.46324    0.678428    2.84908   0.60267  0.0500096

5×5 Array{Float64,2}:
 2.47912    0.0         0.0       0.0      0.0
 0.188601   2.81673     0.0       0.0      0.0
 0.0056648  1.85632     1.11233   0.0      0.0
 1.83752    0.0364813  -0.106363  1.02072  0.0
 1.46324    0.678428    2.84908   0.60267  0.0500096

In [138]:
# NORMA DOS ERROS ! ! !
n = 300
X = randn(n, n)
size_block = 256

A = X*X'
B = copy(A)


L1 = cholesky(A).L
L2 = Cholesky_factorize(A)
L3 = Cholesky_by_blocks(A, size_block)


display(norm(A - L1*L1'))
display(norm(A - L2*L2'))
# display(norm(A - L3*L3'))

PosDefException: [91mPosDefException: matrix is not Hermitian; Cholesky factorization failed.[39m

In [141]:
using LinearAlgebra

function cholesky_by_blocks!(A::Matrix{Float64}, block_size::Int)
    n = size(A, 1)
    for j in 1:block_size:n
        b = min(block_size, n - j + 1)

        # Fatoração do bloco diagonal
        Ajj = @view A[j:j+b-1, j:j+b-1]
        Ljj = cholesky(Ajj).L
        copy!(Ajj, Ljj)

        # Atualização do bloco abaixo
        if j + b <= n
            A21 = @view A[j+b:n, j:j+b-1]
            ldiv!(LowerTriangular(Ljj), A21')

            # Atualização do Schur complement
            A22 = @view A[j+b:n, j+b:n]
            BLAS.syrk!('L', 'N', -1.0, A21, 1.0, A22)
        end
    end

    # Zera a parte superior (para garantir formato lower triangular)
    for i in 1:n
        for j in i+1:n
            A[i, j] = 0.0
        end
    end

    return A
end


cholesky_by_blocks! (generic function with 1 method)

In [144]:
using LinearAlgebra

n = 300
block_size = 64

X = randn(n,n)
A = X * X'
A = Symmetric(A)

B = copy(Matrix(A))  # remove o wrapper Symmetric para o algoritmo inplace

cholesky_by_blocks!(B, block_size)

L_native = cholesky(A).L

println("Erro da fatoração bloca: ", norm(B - L_native))


PosDefException: [91mPosDefException: matrix is not Hermitian; Cholesky factorization failed.[39m

In [128]:
using LinearAlgebra
using BenchmarkTools

x = rand(1000, 1000);
pd = x * x'
A = copy(pd)

# @btime cholesky($pd);
# @btime LAPACK.potrf!('L', $pd);

In [85]:
L = tril(pd)

A - L*L'

2.220446049250313e-16

In [143]:
using LinearAlgebra
using LinearAlgebra.LAPACK
using LinearAlgebra.BLAS

@views function blocked_cholesky!(A::Matrix{Float64}, nb::Int)
    
    n = size(A, 1)
    
    for j = 1:nb:n
        b = min(n - j + 1, nb)
        
        # Passo 1: fatoração Cholesky do bloco A[j:j+b-1, j:j+b-1]
        Ajj = A[j:j+b-1, j:j+b-1]
        uplo = 'L'  # Fatoração inferior
        info = LAPACK.potrf!(uplo, Ajj)

        # Passo 2: triangular solve - TRSM
        if j + b <= n
            A21 = A[j+b:n, j:j+b-1]
            BLAS.trsm!('R', 'L', 'T', 'N', 1.0, Ajj, A21)
        end

        # Passo 3: atualização simétrica - SYRK
        if j + b <= n
            A21 = A[j+b:n, j:j+b-1]
            A22 = A[j+b:n, j+b:n]
            BLAS.syrk!('L', 'N', -1.0, A21, 1.0, A22)
        end
    end
    
    # Zera a parte superior (já que estamos usando 'L')
    for i in 1:n
        for j in i+1:n
            A[i, j] = 0.0
        end
    end

    return A
end


blocked_cholesky! (generic function with 1 method)

In [155]:
x = rand(1000, 1000);
pd = x*x'
A = copy(pd)

@btime cholesky($A)
@btime blocked_cholesky!($pd, 512)

  8.971 ms (6 allocations: 7.63 MiB)
  5.558 ms (5 allocations: 320 bytes)


1000×1000 Array{Float64,2}:
 1.0          0.0           0.0        …       0.0       0.0       0.0
 0.756322  -870.464         0.0                0.0       0.0       0.0
 0.726949    -3.30874      -9.54509            0.0       0.0       0.0
 0.725027    -3.1775        3.00761            0.0       0.0       0.0
 0.726864    -3.75841       3.00913            0.0       0.0       0.0
 0.735788    -3.6435        2.97223    …       0.0       0.0       0.0
 0.760082    -3.38595       2.89862            0.0       0.0       0.0
 0.744626    -3.22696       3.33041            0.0       0.0       0.0
 0.760455    -3.62061       3.43899            0.0       0.0       0.0
 0.73499     -3.3635        2.86311            0.0       0.0       0.0
 0.745124    -3.40303       2.83       …       0.0       0.0       0.0
 0.755999    -3.65663       3.15416            0.0       0.0       0.0
 0.749066    -3.13968       2.9152             0.0       0.0       0.0
 ⋮                                     ⋱         

In [154]:
pd = x * x'
A = copy(pd)

blocked_cholesky!(pd, 2)

display(pd*pd')
display(A)
display(norm(A - pd*pd'))

4×4 Array{Float64,2}:
 2.18165  1.11409   1.2279    2.02837
 1.11409  0.955268  0.417959  1.0724
 1.2279   0.417959  1.06719   1.01608
 2.02837  1.0724    1.01608   1.93202

4×4 Array{Float64,2}:
 2.18165  1.11409   1.2279    2.02837
 1.11409  0.955268  0.417959  1.0724
 1.2279   0.417959  1.06719   1.01608
 2.02837  1.0724    1.01608   1.93202

4.440892098500626e-16